# Data 02
July 12, 2022 - new train/valid IDs.
The split looks good but we're still missing one WSI: TCGA-14-1401-01Z-00-DX1.

In [1]:
import numpy as np
import pandas as pd


In [2]:
FILEPATH='/home/jrm/Adjeroh/Naved/July_Run/TrainTestSplit_82k/'
TRAIN_FILES=['fold0_train.txt','fold1_train.txt','fold2_train.txt','fold3_train.txt','fold4_train.txt']
VALID_FILES=['fold0_test.txt','fold1_test.txt','fold2_test.txt','fold3_test.txt','fold4_test.txt']

In [3]:
def load_df(train,fold):
    filepath = FILEPATH
    if train:
        filepath += TRAIN_FILES[fold]
    else:
        filepath += VALID_FILES[fold]
    df         = pd.read_csv(filepath,header=None)
    df.columns = ['PNG','CANCER']     
    df['CASE'] = df['PNG'].str[0:19]
    df['WSI']  = df['PNG'].str[0:23]
    return df
train0 = load_df(True,0)
train0

,PNG,CANCER,CASE,WSI
0,TCGA-06-0129-01Z-00-DX1_5400_5100.png,0,TCGA-06-0129-01Z-00,TCGA-06-0129-01Z-00-DX1
1,TCGA-06-0129-01Z-00-DX1_5700_6000.png,0,TCGA-06-0129-01Z-00,TCGA-06-0129-01Z-00-DX1
2,TCGA-06-0129-01Z-00-DX1_2400_2100.png,0,TCGA-06-0129-01Z-00,TCGA-06-0129-01Z-00-DX1
3,TCGA-06-0129-01Z-00-DX1_5700_3900.png,0,TCGA-06-0129-01Z-00,TCGA-06-0129-01Z-00-DX1
4,TCGA-06-0129-01Z-00-DX1_4200_3600.png,0,TCGA-06-0129-01Z-00,TCGA-06-0129-01Z-00-DX1
...,...,...,...,...
65595,TCGA-DB-A4XG-01Z-00-DX2_60000_42300.png,5,TCGA-DB-A4XG-01Z-00,TCGA-DB-A4XG-01Z-00-DX2
65596,TCGA-DB-A4XG-01Z-00-DX2_54300_56400.png,5,TCGA-DB-A4XG-01Z-00,TCGA-DB-A4XG-01Z-00-DX2
65597,TCGA-DB-A4XG-01Z-00-DX2_54300_54600.png,5,TCGA-DB-A4XG-01Z-00,TCGA-DB-A4XG-01Z-00-DX2
65598,TCGA-DB-A4XG-01Z-00-DX2_57600_57900.png,5,TCGA-DB-A4XG-01Z-00,TCGA-DB-A4XG-01Z-00-DX2


Counts breakdown by train/valid and by cancer class.

In [4]:
def get_breakdown(df,column):
    breakdown=[]
    for c in range(6):
        rows = df.loc[df['CANCER']==c]
        uniq = rows[column].nunique()
        breakdown.append(uniq)
    return breakdown
def triple(category,df1,df2,column):
    a = df1[column].nunique()
    b = df2[column].nunique()
    breakdown1 = get_breakdown(df1,column)
    breakdown2 = get_breakdown(df2,column)
    print('%15s %6d %6d %6d' % (category,a,b,a+b),end='   ')
    print(breakdown1,breakdown2)
def basic_stats(train,valid):
    triple('Unique CASE:',train,valid,'CASE')
    triple('Unique WSI :',train,valid,'WSI')
    triple('Unique PNG :',train,valid,'PNG')

In [5]:
for fold in range(5):
    print('Fold',fold,'          Train  Valid  Total   [Train Breakdown] [Valid Breakdown]')
    train_df = load_df(True,fold)
    valid_df = load_df(False,fold)
    basic_stats(train_df,valid_df)

Fold 0           Train  Valid  Total   [Train Breakdown] [Valid Breakdown]
   Unique CASE:     83     25    108   [36, 16, 13, 11, 5, 2] [10, 5, 4, 3, 2, 1]
   Unique WSI :    171     45    216   [101, 24, 25, 13, 5, 3] [16, 9, 12, 4, 3, 1]
   Unique PNG :  65600  17370  82970   [38865, 9391, 9273, 4873, 2003, 1195] [6101, 3511, 4677, 1491, 1194, 396]
Fold 1           Train  Valid  Total   [Train Breakdown] [Valid Breakdown]
   Unique CASE:     83     25    108   [36, 16, 13, 11, 5, 2] [10, 5, 4, 3, 2, 1]
   Unique WSI :    160     56    216   [86, 23, 29, 13, 6, 3] [31, 10, 8, 4, 2, 1]
   Unique PNG :  61605  21365  82970   [32935, 8983, 11108, 5000, 2393, 1186] [12031, 3919, 2842, 1364, 804, 405]
Fold 2           Train  Valid  Total   [Train Breakdown] [Valid Breakdown]
   Unique CASE:     83     25    108   [36, 16, 13, 11, 5, 2] [10, 5, 4, 3, 2, 1]
   Unique WSI :    168     48    216   [101, 26, 20, 12, 6, 3] [16, 7, 17, 5, 2, 1]
   Unique PNG :  65500  17470  82970   [39186, 1016

Validate that no case is in the train and valid of the same fold.

In [9]:
problems = 0
for fold in range(5):
    train_df = load_df(True,fold)
    valid_df = load_df(False,fold)
    valid_cases = valid_df['CASE'].unique() 
    for case in valid_cases:
        matching_train_cases = train_df.loc[train_df['CASE']==case]
        if len(matching_train_cases) > 0:
            print("Problem with case",case)
            problems += 1
if problems==0:
    print("No problems")

No problems
